In [1]:
import os
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

os.chdir("C:\\Users\\lijy\\OneDrive - CentraleSupelec\\商联周报\\20190715-20190721")

In [2]:
sales_growth = pd.read_csv("caa_sales_growth.csv")
retailer_sales_growth = pd.read_csv("retailer_sales_growth.csv")
cat2_4wk = pd.read_csv("caa_cate2_4_weeks_uplift_result.csv")

In [3]:
# check number of unique retailers == 18
retailer_sales_growth.retailer_name.nunique()

14

In [4]:
retailer_sales_growth.retailer_name.unique()

array(['海纳星地', '乐城', '安德利', '浙江万客隆', '日照新世纪', '万福隆', '海港超市', '保真超市',
       '宏业绿城', '新天地', '亿丰', '邯郸阳光', '新乐', '三和'], dtype=object)

### Clean data

In [5]:
result_huanbi = pd.read_csv("result_huanbi.csv",encoding='utf_8_sig',sep=';',header=None)
result_tongbi = pd.read_csv("result_tongbi.csv",encoding='utf_8_sig',sep=';',header=None)

In [6]:
# check huanbi
result_huanbi["check_delimiter"] = result_huanbi.iloc[:,0].apply(lambda i: len(i.split(",")))
result_huanbi[result_huanbi["check_delimiter"]>5]

,0,check_delimiter


In [7]:
# check tongbi
result_tongbi["check_delimiter"] = result_tongbi.iloc[:,0].apply(lambda i: len(i.split(",")))
result_tongbi[result_tongbi["check_delimiter"]>5]

,0,check_delimiter
98763,"欣安插座新国标JH810/10A,3米,五金器具,\N,45.00,6927844581035",6


In [8]:
def clean_row(row):
    lst = row.split(",")
    if len(lst) > 5:
        row = row.replace(',', '', 1)
    return row

In [9]:
def clean_result(df):
    df.iloc[:,0] = df.iloc[:,0].apply(clean_row)
    df = df.iloc[:,0].str.split(',', expand=True)
    df.columns = ["商品","二级品类","本周销售额","对比周销售额","条形码"]
    # replace \N
    df["本周销售额"] = df["本周销售额"].apply(lambda i: i.replace("\\N","0"))
    df["对比周销售额"] = df["对比周销售额"].apply(lambda i: i.replace("\\N","0"))
    df["本周销售额"] = df["本周销售额"].astype(float)
    df["对比周销售额"] = df["对比周销售额"].astype(float)
    df["增长"] = df["本周销售额"] - df["对比周销售额"]
    return df

In [10]:
result_huanbi = clean_result(result_huanbi)
result_tongbi = clean_result(result_tongbi)

In [11]:
result_huanbi.to_csv("result_huanbi_adj.csv")
result_tongbi.to_csv("result_tongbi_adj.csv")

### 销售表现

#### for graph

In [12]:
sales_growth_pivot = pd.pivot_table(sales_growth, values = "growth_rate",columns = "kpi_type",index = "store_format_desc").reset_index()

# reorder
sales_growth_pivot['store_format_desc'] = sales_growth_pivot['store_format_desc'].astype('category')
store_order = ["商联","大卖场","大型超市","便利店","小超市/生鲜店"]
sales_growth_pivot['store_format_desc'].cat.reorder_categories(store_order, inplace=True)
sales_growth_pivot.sort_values('store_format_desc', inplace=True)
rate_order = ["store_format_desc","tongbi","huanbi"]
sales_growth_pivot[rate_order]

kpi_type,store_format_desc,tongbi,huanbi
1,商联,-0.041261,0.006842
2,大卖场,1.187456,-0.011568
3,大型超市,-0.112583,-0.002235
0,便利店,-0.046061,0.022403
4,小超市/生鲜店,0.023013,0.006825


#### 增幅排名

In [13]:
@interact
def retailer_rank(store_format=["大卖场","大型超市","便利店","小超市/生鲜店"],kpi_type=["tongbi","huanbi"]):
    growth = retailer_sales_growth[(retailer_sales_growth.kpi_type == kpi_type) & (retailer_sales_growth.retailer_growth_rate > 0)]
    top3 = growth[growth.store_format_desc == store_format].sort_values("retailer_growth_rate",ascending = False)[["retailer_name"]].head(3)
    return top3

interactive(children=(Dropdown(description='store_format', options=('大卖场', '大型超市', '便利店', '小超市/生鲜店'), value='大…

### Sales by category

In [16]:
cat2_sales_growth = pd.read_csv("category_level2_sales_growth.csv")

In [17]:
@interact
def show_top_cat(category=["食品大类","非食大类","百货和其他大类"],kpi_type=["tongbi","huanbi"]):
    df = cat2_sales_growth[(cat2_sales_growth['category_desc1'] == category) & (cat2_sales_growth.kpi_type == kpi_type) & (cat2_sales_growth.category_level2_growth_rate > 0)]
    df_top = df.sort_values(by = "category_level2_growth_rate", ascending = False).head(8)
    return df_top[["category_desc2"]]

interactive(children=(Dropdown(description='category', options=('食品大类', '非食大类', '百货和其他大类'), value='食品大类'), Dro…

In [18]:
# return top products
@interact
def top_products(category=["食品大类","非食大类","百货和其他大类"],kpi_type=["huanbi","tongbi"],rank=(1,8,1)):
    if kpi_type == "tongbi":
        df = result_tongbi
    else:
        df = result_huanbi
    condition = (df["二级品类"] == show_top_cat(category,kpi_type).iloc[rank-1,0]) & (df["增长"]>0) & (df["条形码"] != 'N')
    result = df[condition].sort_values(by="增长",ascending=False).head(4)
    return result[["商品","条形码","二级品类"]]

interactive(children=(Dropdown(description='category', options=('食品大类', '非食大类', '百货和其他大类'), value='食品大类'), Dro…

In [16]:
# 未来四周增长
@interact
def future_4wk(category=["食品大类","非食大类","百货和其他大类"]):
    df = cat2_4wk[(cat2_4wk.category_desc1 == category) & (cat2_4wk.category_level2_rate > 0)]
    df = df.sort_values(by = "category_level2_rate",ascending = False)[["category_desc2","category_level2_rate"]].head(10)
    return df
    

interactive(children=(Dropdown(description='category', options=('食品大类', '非食大类', '百货和其他大类'), value='食品大类'), Out…

### 热门单品榜

In [17]:
pop_df = pd.read_csv("caa_popular_product.csv", encoding='utf_8_sig')
new_df = pd.read_csv("caa_new_product.csv", encoding = 'utf_8_sig')

In [18]:
pop_df.sort_values(by='visitpntr',ascending=False)[["prod_desc","prod_barcode"]].head(10)

,prod_desc,prod_barcode
0,农夫山泉天然水550ml,6921168592555
1,泉阳泉矿泉水,6924613686625
2,康师傅红茶,6922507096369
3,可口550ml冰露水,6954767470573
4,怡宝矿泉水,6901285991233
5,农夫山泉天然水,6921168509256
6,康师傅冰红茶550ml,6940038005018
7,蒙牛100G*8风味酸牛奶原味圆周杯,6934665091117
8,红牛饮料原味型,6920202888883
9,景田百岁山饮用天然矿泉水570ml*24/箱,6922255466476


In [19]:
new_df.sort_values(by='prod_rank',ascending=False)[["prod_desc","prod_barcode"]].head(10)

,prod_desc,prod_barcode
0,璐璐凯帝铝框万向轮拉杆箱ABS+PC-24寸,6943509801126
2,龙江明珠16cm奶锅,6930890200256
1,双枪塑料砧板ZB0041,6921988316157
3,蒙牛227g缔芝特黄油,6923644277215
4,全棉健康夏凉被200*230,6970676130049
5,山润100%菜籽油5L,6941287806906
6,洁柔FACE天然无香8连包抽纸,6914068022123
7,爵百莉英伦香蜂草水润盈彩洗发露600毫升,5016889001705
8,君乐宝800G乐纯婴儿配方奶粉,6922577726777
9,可心柔小碎花360张3层软抽6连包VF1301-6,6957270913056
